In [5]:
#How much you would get if you accept an offer for a position from X company based on number of experiences and location?
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

#read data from file
dataset = pd.read_csv('data/data.csv')

df_orig = dataset.copy(deep=True)

#extract features that we want to consider
df = df_orig[['company','title','yearsofexperience','location', 'totalyearlycompensation']]

# identify invalid data
df.isna().sum()

# delete invalid data
df.dropna()

# one hot encoding for categorical features
categorical_columns = ['title', 'location','company']
for column in categorical_columns:
    tempdf = pd.get_dummies(df[column], prefix=column)
    df = pd.merge(
        left=df,
        right=tempdf,
        left_index=True,
        right_index=True,
    )
    df = df.drop(columns=column)

np.random.seed(2)
n = len(df)
# splitting of data to test(30%) and train(70%)
n_test = int(0.3 * n)
n_train = n - (n_test)

idx = np.arange(n)
# shuffling the data
np.random.shuffle(idx)

df_original_shuffled = df_orig.iloc[idx]
df_shuffled = df.iloc[idx]

df_train = df_shuffled.iloc[:n_train].copy()
df_test = df_shuffled.iloc[n_train:].copy()

df_original_test = df_original_shuffled[n_train:].copy()

#linear regression
df_train_features = df_train.drop(columns=['totalyearlycompensation'])
df_train_labels = df_train['totalyearlycompensation']
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[len(df_train_features.columns)])])
model.compile(optimizer='sgd', loss='mean_squared_error',metrics=[tf.keras.metrics.RootMeanSquaredError()])

model.fit(df_train_features, df_train_labels, epochs=100)

#seperate features and labels for prediction
df_test_features = df_test.drop(columns=['totalyearlycompensation'])
df_test_labels = df_test['totalyearlycompensation']

#prediction
prediction_test=model.predict(df_test_features)

#calculate rmse
error = prediction_test - df_test_labels.values
mse = (error ** 2).mean()
rmse= np.sqrt(mse)

print("The RMSE value on test data is ",rmse)

#print test data with original and predicted yearly compensation
df_original_test['predicated_totalyearlycompensation']=prediction_test
df_original_test[['company','title','yearsatcompany','totalyearlycompensation','predicated_totalyearlycompensation']].sort_values('predicated_totalyearlycompensation', ascending=False).head()




Train on 43850 samples
Epoch 1/100
43850/43850 [==============================] - 3s 68us/sample - loss: 19976497746.9242 - root_mean_squared_error: 141338.2969
Epoch 2/100
43850/43850 [==============================] - 3s 64us/sample - loss: 16883057452.9883 - root_mean_squared_error: 129934.8750
Epoch 3/100
43850/43850 [==============================] - 3s 59us/sample - loss: 16475785705.6752 - root_mean_squared_error: 128358.0078s - loss: 15909480687.3273 - root_mean_squared_error:
Epoch 4/100
43850/43850 [==============================] - 3s 58us/sample - loss: 17666809002.8924 - root_mean_squared_error: 132916.5938
Epoch 5/100
43850/43850 [==============================] - 3s 61us/sample - loss: 21905226340.5318 - root_mean_squared_error: 148004.1406s - los
Epoch 6/100
43850/43850 [==============================] - 3s 59us/sample - loss: 18403970876.3074 - root_mean_squared_error: 135661.2656
Epoch 7/100
43850/43850 [==============================] - 3s 60us/sample - loss: 1803255

43850/43850 [==============================] - 4s 83us/sample - loss: 20656213914.9077 - root_mean_squared_error: 143722.7188s - loss: 23960602549.9234 - root_mea - ETA: 0s - loss: 21072427370.4049 - root_mean_squared_error: 14516
Epoch 53/100
43850/43850 [==============================] - 3s 68us/sample - loss: 15421996803.4678 - root_mean_squared_error: 124185.3125
Epoch 54/100
43850/43850 [==============================] - 2s 57us/sample - loss: 15493137957.3404 - root_mean_squared_error: 124471.4766
Epoch 55/100
43850/43850 [==============================] - 2s 57us/sample - loss: 14649576857.3665 - root_mean_squared_error: 121035.4531
Epoch 56/100
43850/43850 [==============================] - 3s 59us/sample - loss: 19299289124.8558 - root_mean_squared_error: 138921.9688s - loss: 22599217651.8271 - root_m - ETA: 0s - loss: 19288507816.6331 - root_mean_square
Epoch 57/100
43850/43850 [==============================] - 3s 61us/sample - loss: 14990384714.7041 - root_mean_squared_erro

,company,title,yearsatcompany,totalyearlycompensation,predicated_totalyearlycompensation
20778,Netflix,Data Scientist,0.0,375000,400178.15625
13667,Snap,Software Engineer,0.0,198000,357901.28125
48762,Snap,Data Scientist,0.0,385000,348078.21875
37949,Snap,Software Engineer,0.0,188000,346055.71875
52492,Snap,Software Engineer,0.0,191000,346055.71875
